## Import Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle

#Import function to get the covariate matrix that includes spike history from previous bins
from preprocessing_funcs import get_spikes_with_history

#Import decoder functions
import Decoder_funcs
from Decoder_funcs import get_vaf

/opt/anaconda/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.
Using gpu device 3: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5103)
/opt/anaconda/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


## Import Data

**Neural data should be a matrix of size "number of time bins" x "number of neurons", where each entry is the firing rate of a given neuron in a given time bin**

**The output you are decoding should be a matrix of size "number of time bins" x "number of features you are decoding"**

In [2]:
# folder='/Users/jig289/Dropbox/MATLAB/Projects/In_Progress/BMI/Processed_Data/'
folder='/home/jglaser2/Data/DecData/'

with open(folder+'test_data.pickle','rb') as f:
    neural_data,vels_binned=pickle.load(f)

## Define Covariates ##

### User Options

In [3]:
bins_before=14 #How many bins of neural data prior to the output are used for decoding
bins_after=0 #How many bins of neural data after (and including) the output are used for decoding
bins_surrounding=bins_before+bins_after

### Get Covariates

In [4]:
#Function to get the covariate matrix that includes spike history from previous bins
X=get_spikes_with_history(neural_data,bins_before,bins_after)

#Put in "flat" format for XGB and linear
X_flat=X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))

#Set decoding output
y=vels_binned

### Process Covariates

In [5]:
# zscore = lambda x: (x - scipy.stats.nanmean(x)) / scipy.stats.nanstd(x)
def zscore_nan(X,axis):
    X_zscore=(X - np.nanmean(X,axis=axis)) / np.nanstd(X,axis=axis)
    return X_zscore


In [6]:
#Normalize

X=zscore_nan(X,axis=0)
X_flat=zscore_nan(X_flat,axis=0)

# y=stats.zscore(y,axis=0)

## Split into training/testing/validation sets

### User Options

In [7]:
#Set what part of data should be part of the training/testing/validation sets
training_range=[0, 0.8]
testing_range=[0.8, 1]
valid_range=[1,1]

### Split Data

In [8]:
num_examples=X.shape[0]

#Note that each range has a buffer of"bins_before" bins at the beginning, and "bins_after" bins at the end
#This makes it so that the different sets don't include overlapping neural data
training_set=np.arange(np.int(np.round(training_range[0]*num_examples))+bins_before,np.int(np.round(training_range[1]*num_examples))-bins_after)
testing_set=np.arange(np.int(np.round(testing_range[0]*num_examples))+bins_before,np.int(np.round(testing_range[1]*num_examples))-bins_after)
valid_set=np.arange(np.int(np.round(valid_range[0]*num_examples))+bins_before,np.int(np.round(valid_range[1]*num_examples))-bins_after)

#Divide covariates
X_train=X[training_set,:,:] #Subtract X.shape[1] so we don't have overlap in the train/test sets
X_flat_train=X_flat[training_set,:]
y_train=y[training_set,:]

X_test=X[testing_set,:,:] #Subtract X.shape[1] so we don't have overlap in the train/test sets
X_flat_test=X_flat[testing_set,:]
y_test=y[testing_set,:]

X_valid=X[valid_set,:,:] #Subtract X.shape[1] so we don't have overlap in the train/test sets
X_flat_valid=X_flat[valid_set,:]
y_valid=y[valid_set,:]

## Run Decoders

### Wiener Filter (Linear Regression)

In [9]:
#Get predictions
y_test_pred_lin=Decoder_funcs.lin_reg_model(X_flat_train,y_train,X_flat_test)

### XGBoost (Extreme Gradient Boosting)

In [10]:
#Get predictions
y_test_pred_xgb=Decoder_funcs.xgb_model(X_flat_train,y_train,X_flat_test,max_depth=3,num_round=200)

### Simple RNN

In [10]:
#Get predictions
y_test_pred_rnn=Decoder_funcs.SimpleRNN_model(X_train,y_train,X_test,units=400,dropout=0,num_epochs=10)

### GRU (Gated Recurrent Unit)

In [ ]:
#Get predictions
y_test_pred_gru=GRU_model(X_train,y_train,X_test,units=400,dropout=0,num_epochs=10)

### LSTM (Long Short Term Memory)

In [ ]:
#Get predictions
y_test_pred_lstm=LSTM_model(X_train,y_train,X_test,units=400,dropout=0,num_epochs=10)

## Get Metrics

In [16]:
vafs_lin=get_vaf(y_test,y_test_pred_lin)
vafs_lin

[0.73368440278725799, 0.75757097029577536]

In [11]:
vafs_xgb=get_vaf(y_test,y_test_pred_xgb)
vafs_xgb

[0.76302319585941869, 0.78127085225676074]

In [12]:
vafs_rnn=get_vaf(y_test,y_test_pred_rnn)
vafs_rnn

[0.80339951065613802, 0.75997908759960453]

In [ ]:
vafs_gru=get_vaf(y_test,y_test_pred_gru)

In [ ]:
vafs_lstm=get_vaf(y_test,y_test_pred_lstm)

## Make Plots

In [ ]:
#As an example, I plot the first 1000 values of the x velocity (column index 0), both true and predicted with the Wiener filter
fig_x_lin=plt.figure()
plt.plot(y_test[0:1000,0],'b')
plt.plot(y_test_pred_lin[0:1000,0],'r')
#Save figure
# fig_x_lin.savefig('x_velocity_decoding.eps')